In [ ]:
# Parse a fit file
from power_metrics_lib.file_parsers import parse_fit_activity_file

activity_data: list[dict] = parse_fit_activity_file("../tests/files/file.fit")

power_data: list[int] = [a["power"] for a in activity_data]

In [ ]:
# Plot the power data
import matplotlib.pyplot as plt

plt.plot(power_data)
plt.ylabel("watt")
plt.xlabel("duration")
plt.show()

In [ ]:
# Calculate some initial metrics:
import power_metrics_lib.calculate_metrics as pm

np = pm.calculate_normalized_power(power_data)
ap = pm.calculate_average_power(power_data)
print(f"normalized power (np): {np}")
print(f"average power (ap): {ap}")

In [ ]:
# Based on np, we can calculate the intensity_factor given your ftp:
ftp = 236
intensity_factor = pm.calculate_intensity_factor(np, ftp)
print(f"intensity_factor: {intensity_factor}")

In [ ]:
# More metrics:
duration = pm.calculate_duration(power_data)
tss = pm.calculate_training_stress_score(
    normalized_power=np,
    ftp=ftp,
    duration=duration["seconds"],
    intensity_factor=intensity_factor,
)
print(
    f"duration:\n \tin seconds: {duration['seconds']}\n\tin hhmmss: {duration['hh:mm:ss']}"
)
print(f"total stress score: {tss}")

In [ ]:
# Total work and max power:
total_work = pm.calculate_total_work(power_data)
max_power = pm.calculate_max_power(power_data)
print(f"total work: {total_work}")

print(f"max power: {max_power}")

In [ ]:
# Plot elevation:
elevation_data: list[int] = [a["altitude"] for a in activity_data]

plt.plot(elevation_data)
plt.ylabel("altitude")
plt.xlabel("duration")
plt.show()